# <img style="float: right;"  src="images/jp.png" width="200">

# Inverted Pendulum

This document describes the control of an inverted pendulum

![Inverted pendulum](images/InvertedPendulum.png)

We have a pendulum that we want to keep vertical (at angle zero). The pendulum generates a load torque **TL** that can be calculated:

$$T_{L}=m \cdot g \cdot l \cdot sin(\theta)$$

We will use a motor to generate a torque **Tm**, that aims to keep the pendulum straight

## Load Python modules

We will start giving some values to the pendulum properties.

The [calc module](http://localhost:8888/edit/Code/calc.py) is a non standard **Python** module that you can find in the [Code folder](http://localhost:8888/tree/Code)

**Execute** the following code block to load the needed Python modules.

In [ ]:
# We import numpy and calc
import numpy as np
import calc

# Check loaded module version
try:
    print('calc version: ',calc.version)
except:
    print('Error loading the calc module')

## Pendulum parameters

Now we can give the pendulum some parameters and calculate the torque **TL** as function of the angle.

**Execute** the following code block.

In [ ]:
# Data for the pendulum

m = 0.1   # [kg]   Mass
l = 0.1   # [m]    Length
g = 9.81  # [s^-2] Gravity acceleration

# Torque vs angle graph

vAngle = np.arange(-180.0,180.0,1.0)
vTl    = m*g*l*np.sin(np.deg2rad(vAngle))

# Torque graph
calc.plot11(vAngle,vTl,"Torque vs Angle","Angle (deg)","Torque (n*m)")

## Motor parameters

Now we can define a motor to be used with the pendulum.

For this document we will define a motor that has a maximum torque bigger than the maximum torque the pendulum can generate.

We will also consider that the only inertial momentum of the motor is the one of the pendulum

$$J=m \cdot l^2$$

**Execute** the following code block to generate the motor data.

In [ ]:
# Motor data

# We will calculate the motor parameters from typical motor information
# We will only consider the intertial momentum of the pendulum

dcVoltage    = 12     # [V]
stallCurrent = 85     # [A]
stallTorque  = 0.5    # [N*m]
unloadedRPM  = 19300  # [min^-1]

# Calculations of constants of the motor

# Convert unloaded speed from RPM to rad/s
unloadedSpeed = 2.0*np.pi*unloadedRPM/60.0

# Calculate the winding resistance from the stall current
R = dcVoltage/stallCurrent

# Calculate the motor constant
k = stallTorque/stallCurrent

# Inertial momentum
J = m*l*l 

# Show the results
calc.printVar("R",R,"Ohms")
calc.printVar("k",k,"N*m / A")
calc.printVar("J",J,"kg*m^2",sci=False)

## Open loop

We can try to operate the pendulum in open loop

The following figure shows the pendulum and stall motor torques for a giving motor voltage as function of the angle

**Execute** the code block.

In [ ]:
# Define the motor voltage
V = 1.0 #[V]

# Stall motor torque
Tm = V*k/R
vTm=Tm*np.ones(len(vAngle))

# Torque graph

calc.plot1n(vAngle,[vTl,vTm],"Torque vs Angle","Angle (deg)"
             ,"Torque (n*m)",["Pendulum","Motor"])

In the above figure positive pendulum torque, in blue, moves the pendulum to the right (towards more positive angles), whereas the motor torque, when it dominates, move the pendulum to the left (towards more negative angles).

In the above figure you can see that there is only **one stable equilibrium point** at about 155 deg. If the angle increases, the motor will move to **decrease** the angle and, if the angle decreases, the **pendulum** will dominate and **increase** the angle. As the system tries to compensate **against** the angle change, we have **negative feedback**.

Negative feedback gives **static stability**, but does not guarantee **dynamic stability**. 

Note that there is also one **inestable equilibrium point** at abot 25 deg. If the angle **increases**, the pendulum dominates and **increases** the angle enven more. If the angle **decreases**, the motor torque dominates and it **decreases** the angle even more. As the **system** **increases** any deviation from the **equilibrium** point, we have **positive** feedback that makes the **equilibrium** point inestable even from a static analysis point of view.

We can solve the dynamics in open loop by calculating the torque on the pendulum at each time step

$$T=m \cdot g \cdot l \cdot sin(\theta)+\frac{k}{R}(-V-k\cdot \omega)-\mu\cdot\omega$$

We have added a friction term (the last one in the above expression) to be a little more realistic on a practical case. It also helps to damp a little the operation of the system.

**Execute** the following code block to simulate the system in open loop.

In [ ]:
# System dynamics

# We will consider a friction value to damp the system
mu = 2e-3 # [N*m*s]

# Time information
tEnd  = 5.0 #[s]
tStep = 0.001 #[s]
vTime = np.arange(0.0,tEnd,tStep)

# Start condition
startAngle = 170

# State variables
angle = np.deg2rad(startAngle) #rad
speed = 0 #rad/s

# Output vectors
vSpeed  = []
vAngle  = []
vTorque = []

# Do the simulation
# We solve using the Euler method because is easier to
# understand and we only need qualitative results
for time in vTime:
    # Calculate torque
    T = m*g*l*np.sin(angle)+(k/R)*(-V-k*speed)-mu*speed
    # Calculate acceleration, speed and angle
    alpha = T/J
    speedNew = speed + alpha*tStep
    angleNew = angle + speed*tStep
    # Update output data
    vTorque.append(T)
    vSpeed.append(speed)
    vAngle.append(angle)
    # Udate state information
    speed = speedNew
    angle = angleNew
    
# Show angle graph
calc.plot11(vTime,np.rad2deg(vAngle),"Angle in open loop"
            ,"Time (s)","Angle (deg)")


## PID Closed loop operation

Using open loop operation we cannot reach the upward position on the inverted pendulum. Moreover, we have little controlon the dynamics of the system.

The best control of the pendulum requires using a controller that generates the motor drive voltage as function of the state of the system.

There are several option for the control system. One of them is using a PID controller.

![PID controller](images/PID_Motor.png)

In a PID contoller we compare the system output with a goal $G$ value and calculate the error between the state and the goal. As, in our case we want to control the angle, the error will be:

$$\varepsilon=\theta-\theta_{G}$$

Then we generate the input of the system by negative feedback using three elements:

1) Value proportional to the error

2) Value proportions to the time integral of the error

3) Value proportional to the time derivative of the error

So, in our case:

$$V=-P \cdot \varepsilon -I \cdot \int \varepsilon dt -D \cdot \frac{d\varepsilon}{dt}$$

The following code simulates the inverted pendulum and the motor dynamics under PID control.

You can play changing the goal and the P, I and D values and **executing** the code.


In [ ]:
# Closed loop operation

# This time we will solve the dynamics using Runge-Kutta
# That gives a much more exact solution depending on the 
# P, I and D settings of the controller

# Some interesting cases
# P =  5, I =   0, D =  0 Proportional only (Error depends on goal angle)
# P = 20, I =   0, D =  0 Less error but more oscillations
# P =  5, I = 0.5, D =  0 Less error than P=5 only
# P =  5, I = 0.5, D =  5 Reduced oscillations
# P = 50, I =  10, D = 20 Fast and damped

# Goal angle
angleGoal = 20.0

# Proportional constant
P = 50.0

# Integral constant
I = 0.0

# Derivative constant
D = 0.0

# We will consider a friction value to damp the system
mu = 2e-3 # [N*m*s]

# Time information
tEnd  = 5.0 #[s]
tStep = 0.001 #[s]
vTime = np.arange(0.0,tEnd,tStep)

# Start condition
startAngle = -70

# State variables [speed,angle]
x = [0.0,np.deg2rad(startAngle)]

# Output vector
vAngle  = []

# Angle goal in rad
goal = np.deg2rad(angleGoal)

# Initialize integral
integ = 0.0

# Initialize derivative
prevError = 0

# Derivative of the state variables (for Runge-Kutta)
def fderClose(x,t):
    global integ,prevError
    error = x[1]-goal
    
    # Calculate integral term
    integ = integ + error*tStep
  
    # Calculate voltage 
    V = -(P*error+I*integ+D*(error-prevError)/tStep)
    if V > dcVoltage:
       V = dcVoltage
    if V < -dcVoltage:
       V = -dcVoltage
        
    prevError = error
    
    # Calculate torque
    T = m*g*l*np.sin(x[1])+(k/R)*(V-k*x[0])-mu*x[0]
    # Calculate acceleration
    alpha = T/J
    # Return state derivatives
    # [0] d speed / dt = alpha
    # [1] d angle / dt = speed
    return np.array([alpha,x[0]])

# Do the simulation using Runge-Kutta (4th order)
for time in vTime:
    # Store data
    vAngle.append(x[1])
    
    # Update state using Runge-Kutta
    x = calc.rk4(x,time,fderClose,tStep)
    
# Goal value  
vGoal = angleGoal*np.ones(len(vTime))
    
# Show graphs
calc.plot1n(vTime,[np.rad2deg(vAngle),vGoal]
             ,"PID operation"
             ,"Time (s)","Angle (deg)")




<BR><BR>

## Document information

Copyright © Vicente Jiménez (2018-2019)

Updated on 14/4/2019

This work is licensed under a [Creative Common Attribution-ShareAlike 4.0 International license](http://creativecommons.org/licenses/by-sa/4.0/). 

<img  src="images/cc_sa.png" width="200">